In [28]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime, timedelta, timezone
import pytz
import scipy.stats
from dateutil.relativedelta import relativedelta

import warnings

# Suppress the specific warnings
warnings.filterwarnings("ignore")

today = datetime.utcnow()

from api_keys import espn_link

In [29]:
homeOnlySked = pd.read_csv('data/sked.csv', index_col=False)
homeOnlySked

completeSked = pd.read_csv('data/sked_full.csv', index_col=False)
completeSked

team_names = pd.read_csv('data/team_names.csv', index_col=False)
team_names

file_name = f"data/allG_df_fp-{today.strftime('%Y-%m-%d')}.csv"
allG_df = pd.read_csv(file_name, index_col=False)
allG_df

file_name = f"data/all_df_fp-{today.strftime('%Y-%m-%d')}.csv"
all_df = pd.read_csv(file_name, index_col=False)
all_df

team_names_only = team_names[['abbreviation', 'name']]

all_df_forwards =  all_df.loc[all_df['position'] != 'D']
all_df_defense =  all_df.loc[all_df['position'] == 'D']

all_df_centers =  all_df.loc[all_df['secondaryPosition'] == 'C']
all_df_wingers =  all_df.loc[all_df['secondaryPosition'] == 'W']

file_name = f"data/summary_stats-{today.strftime('%Y-%m-%d')}.csv"
player_bios = pd.read_csv(file_name, index_col=False)

# file_name = f"data/rankings_worksheet_{(today-timedelta(days=6)).strftime('%Y-%m-%d')}.xlsx"
# rankings = pd.read_excel(file_name)

In [30]:
startDate = '2023-11-27'
endDate = '2023-11-29'

In [31]:
lookahead_sked = homeOnlySked.loc[(homeOnlySked['gameDate'] >= startDate) & (homeOnlySked['gameDate'] <= endDate)]
lookahead_sked.sort_values('gameTime')

,gameID,gameDate,gameTime,awayTeam,homeTeam,gameDT,winningGoalie
7,2023020322,2023-11-27,Monday 07:00 PM,BUF,NYR,2023-11-28 00:00:00,0
1118,2023020321,2023-11-27,Monday 07:00 PM,BOS,CBJ,2023-11-28 00:00:00,0
1078,2023020323,2023-11-27,Monday 07:00 PM,FLA,OTT,2023-11-28 00:00:00,0
214,2023020324,2023-11-27,Monday 09:00 PM,TBL,COL,2023-11-28 02:00:00,0
991,2023020325,2023-11-27,Monday 09:30 PM,VGK,CGY,2023-11-28 02:30:00,0
1283,2023020326,2023-11-27,Monday 10:30 PM,WSH,SJS,2023-11-28 03:30:00,0
501,2023020329,2023-11-28,Tuesday 07:00 PM,FLA,TOR,2023-11-29 00:00:00,0
953,2023020327,2023-11-28,Tuesday 07:00 PM,NYI,NJD,2023-11-29 00:00:00,0
584,2023020328,2023-11-28,Tuesday 07:30 PM,CAR,PHI,2023-11-29 00:30:00,0
790,2023020331,2023-11-28,Tuesday 08:00 PM,PIT,NSH,2023-11-29 01:00:00,0


In [32]:
teams_data = {}

for index, row in lookahead_sked.iterrows():

    home_team = row['homeTeam']
    away_team = row['awayTeam']

    if home_team not in teams_data:
        teams_data[home_team] = {'played': 0, 'home': 0, 'away': 0}
    if away_team not in teams_data:
        teams_data[away_team] = {'played': 0, 'home': 0, 'away': 0}

    teams_data[home_team]['played'] += 1
    teams_data[away_team]['played'] += 1
    teams_data[home_team]['home'] += 1
    teams_data[away_team]['away'] += 1

# Create a DataFrame
teams_df = pd.DataFrame.from_dict(teams_data, orient='index')
teams_df = teams_df.sort_values(by=['played', 'home'], ascending=False)

for index, row in teams_df.iterrows():
    abbrev = index
    full_name = team_names.loc[team_names['abbreviation'] == abbrev]['name'].iloc[0]
    teams_df.at[index, 'team_name'] = full_name
    
teams_df

,played,home,away,team_name
NYR,2,2,0,New York Rangers
CBJ,2,2,0,Columbus Blue Jackets
WSH,2,0,2,Washington Capitals
TBL,2,0,2,Tampa Bay Lightning
FLA,2,0,2,Florida Panthers
VGK,2,0,2,Vegas Golden Knights
LAK,1,1,0,Los Angeles Kings
VAN,1,1,0,Vancouver Canucks
COL,1,1,0,Colorado Avalanche
WPG,1,1,0,Winnipeg Jets


In [33]:
team_check = teams_df['team_name'].to_list()

nhl_teams = [
    'New Jersey Devils', 'New York Islanders', 'New York Rangers',
    'Philadelphia Flyers', 'Pittsburgh Penguins', 'Boston Bruins',
    'Buffalo Sabres', 'Montréal Canadiens', 'Ottawa Senators',
    'Toronto Maple Leafs', 'Carolina Hurricanes', 'Florida Panthers',
    'Tampa Bay Lightning', 'Washington Capitals', 'Chicago Blackhawks',
    'Detroit Red Wings', 'Nashville Predators', 'St. Louis Blues',
    'Calgary Flames', 'Colorado Avalanche', 'Edmonton Oilers',
    'Vancouver Canucks', 'Anaheim Ducks', 'Dallas Stars',
    'Los Angeles Kings', 'San Jose Sharks', 'Columbus Blue Jackets',
    'Minnesota Wild', 'Winnipeg Jets', 'Arizona Coyotes',
    'Vegas Golden Knights', 'Seattle Kraken'
]

# # teams_df.index.values

for team in nhl_teams:

    if team not in team_check:
        print(team)

In [34]:
all_df_even = all_df.copy()
all_df_even['evenGoals'] = all_df_even['goals'] - all_df_even['powerPlayGoals'] - all_df_even['shorthandedGoals']
all_df_even['powerPlayAssists'] = all_df_even['powerPlayPoints'] - all_df_even['powerPlayGoals']
all_df_even['shorthandedAssists'] = all_df_even['shPoints'] - all_df_even['shorthandedGoals']
all_df_even['evenAssists'] = all_df_even['assists'] - all_df_even['powerPlayAssists'] - all_df_even['shorthandedAssists']
all_df_even['evenPoints'] = all_df_even['assists'] + all_df_even['goals']
all_df_even['evenToi'] = all_df_even['toi'] - all_df_even['powerPlayToi'] - all_df_even['shorthandedToi']

all_df_even

,playerId,sweaterNumber,name,position,goals,assists,points,plusMinus,pim,hits,...,gameId,gamesPlayed,fantasyPoints,specialTeams,evenGoals,powerPlayAssists,shorthandedAssists,evenAssists,evenPoints,evenToi
0,8478178,43,D. Raddysh,D,0,0,0,1,0,1,...,2023020001,1,0.9,0,0,0,0,0,0,1060
1,8475177,44,C. de Haan,D,0,0,0,0,0,2,...,2023020001,1,0.7,0,0,0,0,0,0,649
2,8480246,48,N. Perbix,D,0,0,0,-1,2,0,...,2023020001,1,0.6,0,0,0,0,0,0,702
3,8475167,77,V. Hedman,D,0,1,1,-1,0,0,...,2023020001,1,1.9,0,0,0,0,1,1,981
4,8478416,81,E. Cernak,D,0,0,0,0,0,6,...,2023020001,1,1.3,0,0,0,0,0,0,884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11509,8476460,55,M. Scheifele,C,0,1,1,-1,0,1,...,2023020319,1,1.9,1,0,1,0,0,1,1173
11510,8475799,62,N. Niederreiter,R,0,0,0,1,0,2,...,2023020319,1,0.5,0,0,0,0,0,0,854
11511,8479536,71,A. Jonsson-Fjallby,L,0,0,0,0,0,2,...,2023020319,1,0.3,0,0,0,0,0,0,469
11512,8478398,81,K. Connor,L,0,1,1,-2,0,0,...,2023020319,1,3.1,1,0,1,0,0,1,1261


In [35]:
def opposition_stats(df):
    
    opponent_stats = df.groupby('opponent').agg({
        'toi': 'sum',
        'assists': 'sum',
        'goals': 'sum',
        'shots': 'sum',
        'hits': 'sum',
        'blockedShots': 'sum',
    #     'fantasyPoints': ['sum', ('mean', lambda x: round(x.mean(), 2))],
        'fantasyPoints': 'sum',
        'gameDate': 'nunique',
        'powerPlayPoints': 'sum'
        # Add more columns as needed
    })

    opponent_stats['FPAPG'] = round(opponent_stats['fantasyPoints'] / opponent_stats['gameDate'], 2)
    opponent_stats['FPAPG_Rank'] = opponent_stats['FPAPG'].rank(ascending=False, method='min').astype(int)

    # opponent_stats['BPG'] = round(opponent_stats['blocked'] / opponent_stats['date'], 2)
    # opponent_stats['BPG_Rank'] = opponent_stats['BPG'].rank(ascending=True, method='min').astype(int)

    opponent_stats = opponent_stats.rename_axis('abbreviation')

    opponent_stats = pd.merge(opponent_stats, team_names_only, on='abbreviation')

    # opponent_stats['powerPlayPoints'] = opponent_stats['powerPlayGoals'] + opponent_stats['powerPlayAssists']

    # opponent_stats = opponent_stats.drop(columns=['powerPlayGoals', 'powerPlayAssists'])

    return opponent_stats

In [36]:
opp_all = opposition_stats(all_df)
opp_forward = opposition_stats(all_df_forwards)
opp_defense = opposition_stats(all_df_defense)
opp_centers = opposition_stats(all_df_centers)
opp_wingers = opposition_stats(all_df_wingers)

opp_even = opposition_stats(all_df_even)

opp_centers

,abbreviation,toi,assists,goals,shots,hits,blockedShots,fantasyPoints,gameDate,powerPlayPoints,FPAPG,FPAPG_Rank,name
0,ANA,117328,47,25,226,82,56,166.3,21,21,7.92,16,Anaheim Ducks
1,ARI,108660,33,23,238,113,56,149.1,20,13,7.46,23,Arizona Coyotes
2,BOS,119169,30,22,234,128,66,147.2,20,7,7.36,25,Boston Bruins
3,BUF,104334,40,24,209,126,49,150.5,21,8,7.17,28,Buffalo Sabres
4,CAR,101229,43,23,161,74,102,173.0,20,16,8.65,7,Carolina Hurricanes
5,CBJ,117394,44,31,257,60,72,177.7,22,5,8.08,12,Columbus Blue Jackets
6,CGY,112021,41,24,219,83,63,158.7,21,13,7.56,21,Calgary Flames
7,CHI,108244,47,32,251,123,56,185.9,19,17,9.78,2,Chicago Blackhawks
8,COL,108470,37,26,191,82,64,155.8,20,12,7.79,18,Colorado Avalanche
9,DAL,107629,33,25,218,114,76,157.7,19,6,8.30,9,Dallas Stars


In [37]:
# g_last_ten_df_team = pd.DataFrame()

# for index, row in team_names_only.iterrows():
#     temp_all = allG_df.loc[allG_df['team'] == row['abbreviation']].sort_values('gameDate', ascending=False)
#     five_dates = temp_all['gameDate'].unique()[:10]
#     temp_filtered = temp_all[temp_all['gameDate'].isin(five_dates)]
#     g_last_ten_df_team = pd.concat([g_last_ten_df_team, temp_filtered])
    
# g_last_ten_df_team

# gopponent_stats_10 = g_last_ten_df_team.groupby('opponent').agg({
#     'toi': 'sum',
#     'goalsAgainst': 'sum',
#     'shots': 'sum',
#     'fantasyPoints': 'sum',
#     'gameDate': 'nunique',
#     'evSaves': 'sum',
#     'evShots': 'sum',
#     'shutout': 'sum',
#     'ppSaves': 'sum',
#     'ppShots': 'sum'
# })

# gopponent_stats_10['FPAPG'] = round(gopponent_stats_10['fantasyPoints'] / gopponent_stats_10['gameDate'], 2)
# gopponent_stats_10['FPAPG_Rank'] = gopponent_stats_10['FPAPG'].rank(ascending=False, method='min').astype(int)

# gopponent_stats_10 = gopponent_stats_10.rename_axis('abbreviation')

# gopponent_stats_10 = pd.merge(gopponent_stats_10, team_names_only, on='abbreviation')

# gopponent_stats_10

In [38]:
gopponent_stats = allG_df.groupby('opponent').agg({
    'toi': 'sum',
    'goalsAgainst': 'sum',
    'shots': 'sum',
#     'fantasyPoints': ['sum', ('mean', lambda x: round(x.mean(), 2))],
    'fantasyPoints': 'sum',
    'gameDate': 'nunique',
    'evSaves': 'sum',
    'evShots': 'sum',
    'shutout': 'sum',
    'ppSaves': 'sum',
    'ppShots': 'sum'
    # Add more columns as needed
})

gopponent_stats['FPAPG'] = round(gopponent_stats['fantasyPoints'] / gopponent_stats['gameDate'], 2)
gopponent_stats['FPAPG_Rank'] = gopponent_stats['FPAPG'].rank(ascending=False, method='min').astype(int)

# gopponent_stats['SPG'] = round(gopponent_stats['shots'] / gopponent_stats['date'], 2)
# gopponent_stats['SPG_Rank'] = gopponent_stats['SPG'].rank(ascending=False, method='min').astype(int)

gopponent_stats = gopponent_stats.rename_axis('abbreviation')

gopponent_stats = pd.merge(gopponent_stats, team_names_only, on='abbreviation')

# gopponent_stats['powerPlayPoints'] = gopponent_stats['powerPlayGoals'] + gopponent_stats['powerPlayAssists']

# gopponent_stats = gopponent_stats.drop(columns=['powerPlayGoals', 'powerPlayAssists', 'id'])

gopponent_stats

,abbreviation,toi,goalsAgainst,shots,fantasyPoints,gameDate,evSaves,evShots,shutout,ppSaves,ppShots,FPAPG,FPAPG_Rank,name
0,ANA,75460,55,604,50.8,21,448,487,0,80,94,2.42,13,Anaheim Ducks
1,ARI,72101,61,542,23.2,20,396,439,1,77,95,1.16,20,Arizona Coyotes
2,BOS,71184,64,624,10.0,20,437,486,0,102,117,0.50,28,Boston Bruins
3,BUF,75360,55,580,47.0,21,427,471,1,85,95,2.24,15,Buffalo Sabres
4,CAR,72368,64,670,32.2,20,501,546,1,86,101,1.61,18,Carolina Hurricanes
5,CBJ,79246,60,661,71.2,22,488,541,2,99,106,3.24,4,Columbus Blue Jackets
6,CGY,76011,56,666,66.0,21,516,561,1,80,88,3.14,5,Calgary Flames
7,CHI,67964,46,523,63.4,19,403,443,2,57,63,3.34,3,Chicago Blackhawks
8,COL,71441,68,646,13.6,20,450,497,3,109,125,0.68,26,Colorado Avalanche
9,DAL,68464,61,582,16.2,19,423,467,1,79,92,0.85,24,Dallas Stars


In [39]:
player_bios.loc[player_bios['name'] == 'A. Ovechkin']

,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,missedGames,gamesRemaining,fantasyPointsRemain,fullName,birthDate,shootsCatches,height,weight,roster_percent,default_pos,espnId
124,8471214,A. Ovechkin,WSH,L,20711,17,31.3,5.44,1.84,0,65,119.6,Alex Ovechkin,1985-09-17,R,75,238,99.322029,LW,3101


In [40]:
team_names_only

last_five_df_team = pd.DataFrame()

for index, row in team_names_only.iterrows():
    temp_all = all_df.loc[all_df['team'] == row['abbreviation']].sort_values('gameDate', ascending=False)
    five_dates = temp_all['gameDate'].unique()[:5]
    temp_filtered = temp_all[temp_all['gameDate'].isin(five_dates)]
    last_five_df_team = pd.concat([last_five_df_team, temp_filtered])
    
last_five_df_team

,playerId,sweaterNumber,name,position,goals,assists,points,plusMinus,pim,hits,...,team,opponent,secondaryPosition,tertiaryPosition,gameDate,gameTime,gameId,gamesPlayed,fantasyPoints,specialTeams
10126,8475752,71,T. Pitlick,C,1,0,1,2,0,2,...,NYR,BOS,C,F,2023-11-25 00:00:00,Saturday 01:00 PM,2023020308,1,2.3,0
10118,8482109,13,A. Lafrenière,L,0,0,0,0,0,1,...,NYR,BOS,W,F,2023-11-25 00:00:00,Saturday 01:00 PM,2023020308,1,0.5,0
10127,8476459,93,M. Zibanejad,C,0,1,1,1,0,0,...,NYR,BOS,C,F,2023-11-25 00:00:00,Saturday 01:00 PM,2023020308,1,2.0,0
10125,8482157,50,W. Cuylle,L,0,0,0,0,0,4,...,NYR,BOS,W,F,2023-11-25 00:00:00,Saturday 01:00 PM,2023020308,1,1.0,0
10124,8476918,26,J. Vesey,L,1,1,2,2,0,1,...,NYR,BOS,W,F,2023-11-25 00:00:00,Saturday 01:00 PM,2023020308,1,4.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8827,8479379,54,G. Smith,R,1,1,2,2,0,2,...,SJS,STL,W,F,2023-11-16 00:00:00,Thursday 10:30 PM,2023020250,1,4.0,0
8828,8475798,64,M. Granlund,C,0,0,0,1,0,0,...,SJS,STL,C,F,2023-11-16 00:00:00,Thursday 10:30 PM,2023020250,1,1.1,0
8829,8474884,68,M. Hoffman,C,2,0,2,1,2,0,...,SJS,STL,C,F,2023-11-16 00:00:00,Thursday 10:30 PM,2023020250,1,4.8,0
8814,8477335,4,K. Burroughs,D,0,0,0,3,2,2,...,SJS,STL,D,D,2023-11-16 00:00:00,Thursday 10:30 PM,2023020250,1,0.9,0


In [41]:
def summary_statistics(df):
    temp_build = df.groupby('playerId').agg({
        'name': 'first',
        'team': 'last',
        'position': 'first',
        'toi': 'sum',
        'gamesPlayed': 'sum',
        'goals': 'sum',
        'assists': 'sum',
        'specialTeams': 'sum',
        'shots': 'sum',
        'hits': 'sum',
        'blockedShots': 'sum',
        'powerPlayToi': 'sum',
        'shorthandedToi': 'sum',
        'plusMinus': 'sum',
        'fantasyPoints': 'sum',
        'secondaryPosition': 'first',
        'tertiaryPosition': 'first'
    }).reset_index()



    temp_build['FPP60'] = (temp_build['fantasyPoints'] / temp_build['toi'] * 3600).round(2)
    temp_build['FPPG'] = (temp_build['fantasyPoints'] / temp_build['gamesPlayed']).round(2)

    temp_build = temp_build.sort_values(by='FPPG', ascending=False)
    
    return temp_build

In [42]:


top_players_by_team = summary_statistics(last_five_df_team).groupby('team').apply(lambda x: x.nlargest(5, 'fantasyPoints'))
top_players_by_team.reset_index(level='team', drop=True, inplace=True)
top_players_by_team

for index, row in top_players_by_team.iterrows():
    top_players_by_team.at[index, 'available'] = 100 - round(player_bios.loc[player_bios['playerId'] == row['playerId']]['roster_percent'].iloc[0], 2)

top_players_by_team

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,...,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG,available
638,8482745,M. McTavish,ANA,C,4950,5,3,2,2,12,...,4,423,7,-4,12.3,C,F,8.95,2.46,22.78
323,8478366,F. Vatrano,ANA,R,5533,5,2,2,1,21,...,2,444,646,-3,10.9,W,F,7.09,2.18,9.24
96,8475462,R. Gudas,ANA,D,5936,5,1,0,0,6,...,9,6,957,-1,8.6,D,D,5.22,1.72,47.87
532,8480950,I. Lyubushkin,ANA,D,5560,5,0,0,0,3,...,13,13,835,-4,7.7,D,D,4.99,1.54,96.89
402,8479368,M. Jones,ANA,L,4212,5,2,1,0,11,...,1,187,0,0,7.4,W,F,6.32,1.48,99.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52,8474590,J. Carlson,WSH,D,7856,5,0,4,0,15,...,13,1063,762,6,12.2,D,D,5.59,2.44,2.22
6,8471214,A. Ovechkin,WSH,L,5963,5,3,0,0,17,...,5,1590,0,2,11.9,W,F,7.18,2.38,0.68
523,8480873,R. Sandin,WSH,D,7613,5,0,2,0,7,...,14,657,230,2,10.3,D,D,4.87,2.06,80.12
399,8479359,B. Malenstyn,WSH,L,4815,5,1,1,0,8,...,9,17,643,2,9.5,W,F,7.10,1.90,99.81


In [43]:

worst_five = summary_statistics(last_five_df_team).groupby('team').apply(lambda x: x.nsmallest(10, 'fantasyPoints'))
worst_five.reset_index(level='team', drop=True, inplace=True)

for index, row in worst_five.iterrows():
    worst_five.at[index, 'available'] = 100 - round(player_bios.loc[player_bios['playerId'] == row['playerId']]['roster_percent'].iloc[0], 2)
    
worst_five = worst_five.loc[worst_five['available'] <= 25]    

worst_five

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,...,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG,available
467,8480039,M. Necas,CAR,C,5310,5,1,1,0,14,...,1,993,0,-3,5.2,C,F,3.53,1.04,16.69
301,8478013,J. Walman,DET,D,3346,3,0,1,1,7,...,8,405,368,3,6.2,D,D,6.67,2.07,17.92
246,8477493,A. Barkov,FLA,C,3422,3,0,1,0,6,...,1,731,380,0,2.5,C,F,2.63,0.83,2.53
295,8477986,B. Montour,FLA,D,5858,4,0,0,0,7,...,5,951,137,-1,3.5,D,D,2.15,0.88,11.71
413,8479400,P. Dubois,LAK,C,4520,5,1,2,0,10,...,2,612,6,3,6.2,C,F,4.94,1.24,17.28
454,8480002,N. Hischier,NJD,C,895,1,1,1,0,2,...,0,14,20,2,3.4,C,F,13.68,3.40,14.99
253,8477500,B. Horvat,NYI,C,5849,5,0,2,2,15,...,0,894,74,0,5.0,C,F,3.08,1.00,6.36
113,8475765,V. Tarasenko,OTT,R,4885,5,0,2,0,9,...,4,572,1,-2,5.3,W,F,3.91,1.06,17.62
650,8483808,A. Kuzmenko,VAN,L,2803,3,0,0,0,2,...,0,495,0,-2,0.2,W,F,0.26,0.07,22.90
458,8480012,E. Pettersson,VAN,C,6058,5,0,2,0,10,...,3,1053,521,0,5.1,C,F,3.03,1.02,0.15


In [44]:
team_names_only


last_five_df = pd.DataFrame()

for index, row in team_names_only.iterrows():
    temp_all = allG_df.loc[allG_df['team'] == row['abbreviation']].sort_values('gameDate', ascending=False)
    five_dates = temp_all['gameDate'].unique()[:5]
    temp_filtered = temp_all[temp_all['gameDate'].isin(five_dates)]
    last_five_df = pd.concat([last_five_df, temp_filtered])
    
last_five_df

,playerId,sweaterNumber,name,position,pim,goalsAgainst,toi,team,opponent,gameDate,...,win,saves,shots,evSaves,evShots,ppSaves,ppShots,gamesPlayed,fantasyPoints,decisionType
595,8471734,32,J. Quick,G,0,4,3568,NYR,BOS,2023-11-25 00:00:00,...,4,27,31,22,25,3,4,1,1.4,W
629,8478048,31,I. Shesterkin,G,0,1,3593,NYR,PHI,2023-11-24 00:00:00,...,4,36,37,23,24,13,13,1,9.2,W
665,8471734,32,J. Quick,G,0,0,3600,NYR,PIT,2023-11-22 00:00:00,...,4,32,32,26,26,5,5,1,13.4,W
654,8478048,31,I. Shesterkin,G,0,4,3537,NYR,DAL,2023-11-20 00:00:00,...,0,30,34,26,29,3,4,1,-2.0,L
559,8478048,31,I. Shesterkin,G,0,3,3600,NYR,NJD,2023-11-18 00:00:00,...,4,30,33,28,30,0,1,1,4.0,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587,8478406,29,M. Blackwood,G,0,2,3881,SJS,MTL,2023-11-24 00:00:00,...,1,22,24,20,22,2,2,1,1.4,OTL
627,8478039,36,K. Kahkonen,G,0,4,1200,SJS,SEA,2023-11-22 00:00:00,...,0,13,17,12,16,1,1,1,-5.4,L
626,8478406,29,M. Blackwood,G,0,3,2400,SJS,SEA,2023-11-22 00:00:00,...,0,10,13,5,7,5,6,1,-4.0,L
658,8478406,29,M. Blackwood,G,0,3,3525,SJS,VAN,2023-11-20 00:00:00,...,0,34,37,27,29,6,6,1,0.8,L


In [45]:
summary_stats_last5G = last_five_df.groupby('playerId').agg({
    'name': 'first',
    'team': 'last',
    'position': 'first',
    'toi': 'sum',
    'gamesPlayed': 'sum',
    'saves': 'sum',
    'shots': 'sum',
    'start': 'sum',
    'shutout': 'sum',
    'fantasyPoints': 'sum'
}).reset_index()

summary_stats_last5G.sort_values('fantasyPoints', ascending=False).head(25)
summary_stats_last5G

for index, row in summary_stats_last5G.iterrows():
    summary_stats_last5G.at[index, 'available'] = 100 - round(player_bios.loc[player_bios['playerId'] == row['playerId']]['roster_percent'].iloc[0], 2)

summary_stats_last5G['FPP60'] = round(summary_stats_last5G['fantasyPoints'] / summary_stats_last5G['toi'] *3600, 2)
summary_stats_last5G

,playerId,name,team,position,toi,gamesPlayed,saves,shots,start,shutout,fantasyPoints,available,FPP60
0,8470594,M. Fleury,MIN,G,7389,2,50,62,2,0,-13.0,86.21,-6.33
1,8471734,J. Quick,NYR,G,7168,2,59,63,2,1,14.8,69.62,7.43
2,8473503,J. Reimer,DET,G,3898,1,29,34,1,0,-3.2,91.74,-2.96
3,8473575,S. Varlamov,NYI,G,7475,2,57,63,2,0,4.4,77.79,2.12
4,8474593,J. Markstrom,CGY,G,11089,3,94,106,3,0,-0.2,48.60,-0.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,8481035,S. Ersson,PHI,G,7500,2,45,47,2,1,16.0,99.39,7.68
64,8481611,P. Kochetkov,CAR,G,9485,3,55,62,2,0,1.0,71.21,0.38
65,8482221,D. Levi,BUF,G,6259,2,43,50,1,0,-4.4,79.16,-2.53
66,8482411,H. Shepard,WSH,G,3600,1,36,37,1,0,9.2,99.95,9.20


In [46]:
top_players_by_team

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,...,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG,available
638,8482745,M. McTavish,ANA,C,4950,5,3,2,2,12,...,4,423,7,-4,12.3,C,F,8.95,2.46,22.78
323,8478366,F. Vatrano,ANA,R,5533,5,2,2,1,21,...,2,444,646,-3,10.9,W,F,7.09,2.18,9.24
96,8475462,R. Gudas,ANA,D,5936,5,1,0,0,6,...,9,6,957,-1,8.6,D,D,5.22,1.72,47.87
532,8480950,I. Lyubushkin,ANA,D,5560,5,0,0,0,3,...,13,13,835,-4,7.7,D,D,4.99,1.54,96.89
402,8479368,M. Jones,ANA,L,4212,5,2,1,0,11,...,1,187,0,0,7.4,W,F,6.32,1.48,99.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52,8474590,J. Carlson,WSH,D,7856,5,0,4,0,15,...,13,1063,762,6,12.2,D,D,5.59,2.44,2.22
6,8471214,A. Ovechkin,WSH,L,5963,5,3,0,0,17,...,5,1590,0,2,11.9,W,F,7.18,2.38,0.68
523,8480873,R. Sandin,WSH,D,7613,5,0,2,0,7,...,14,657,230,2,10.3,D,D,4.87,2.06,80.12
399,8479359,B. Malenstyn,WSH,L,4815,5,1,1,0,8,...,9,17,643,2,9.5,W,F,7.10,1.90,99.81


In [47]:
df = lookahead_sked.copy()

df.rename(columns={'homeTeam': 'focusTeam', 'gameDate': 'date'}, inplace=True)
df_temp = df[['gameID', 'date', 'awayTeam', 'focusTeam', 'gameTime', 'gameDT']]
df_temp.rename(columns={'focusTeam': 'opponent', 'date': 'date', 'awayTeam': 'focusTeam'}, inplace=True)
df.rename(columns={'awayTeam': 'opponent'}, inplace=True)
df['H/A'] = 'H'
df_temp['H/A'] = 'A'
df = pd.concat([df, df_temp])
df.drop(columns=['winningGoalie'], inplace=True)

sked_stats = df
sked_stats.reset_index(inplace=True)
sked_stats = sked_stats.sort_values('gameTime')

sked_stats = sked_stats[['gameID', 'date', 'gameTime', 'focusTeam', 'H/A', 'opponent']]
# sked_stats['opponent'] = sked_stats.groupby('focusTeam')['opponent'].sum()
sked_stats


for index, row in sked_stats.iterrows():
    d = opp_defense.loc[opp_defense['abbreviation'] == row['opponent']]['FPAPG'].iloc[0]
#     drank = D.loc[D.index == row['focus']]['FPAPG_Rank'][0]
    c = opp_centers.loc[opp_centers['abbreviation'] == row['opponent']]['FPAPG'].iloc[0]
#     crank = C.loc[C.index == row['focus']]['FPAPG_Rank'][0]
    w = opp_wingers.loc[opp_wingers['abbreviation'] == row['opponent']]['FPAPG'].iloc[0]
#     wrank = W.loc[W.index == row['focus']]['FPAPG_Rank'][0]
    g = gopponent_stats.loc[gopponent_stats['abbreviation'] == row['opponent']]['FPAPG'].iloc[0]
    
    sked_stats.at[index, 'D_FPAPG'] = d
#     sked_stats.at[index, 'D_RANK'] = drank
    sked_stats.at[index, 'C_FPAPG'] = c
#     sked_stats.at[index, 'C_RANK'] = crank
    sked_stats.at[index, 'W_FPAPG'] = w
#     sked_stats.at[index, 'W_RANK'] = wrank
    sked_stats.at[index, 'G_FPAPG'] = g
    
sked_summary = sked_stats.groupby('focusTeam').sum().sort_values('D_FPAPG', ascending=False)
sked_summary['D_rank'] = sked_summary['D_FPAPG'].rank(ascending=False, method='first').astype(int)
sked_summary['C_rank'] = sked_summary['C_FPAPG'].rank(ascending=False, method='first').astype(int)
sked_summary['W_rank'] = sked_summary['W_FPAPG'].rank(ascending=False, method='first').astype(int)
sked_summary['G_rank'] = sked_summary['G_FPAPG'].rank(ascending=False, method='first').astype(int)
sked_summary

sked_summary.sort_values('D_rank', ascending=True, inplace=True)
sked_summary

d_adds = pd.DataFrame()
for index, row in sked_summary.iterrows():
    temp = top_players_by_team.loc[(top_players_by_team['team'] == index) & (top_players_by_team['position'] == 'D')]
    temp = temp.loc[temp['available'] >= 25]
    temp['D_rank'] = row['D_rank'].astype(int)
    d_adds = pd.concat([d_adds, temp])
    if row['D_rank'] == 5:
        break

sked_summary.sort_values('C_rank', ascending=True, inplace=True)
c_adds = pd.DataFrame()
for index, row in sked_summary.iterrows():
    temp = top_players_by_team.loc[(top_players_by_team['team'] == index) & (top_players_by_team['secondaryPosition'] == 'C')]
    temp = temp.loc[temp['available'] >= 25]
    temp['C_rank'] = row['C_rank'].astype(int)
    c_adds = pd.concat([c_adds, temp])
    if row['C_rank'] == 5:
        break
        
sked_summary.sort_values('W_rank', ascending=True, inplace=True)
w_adds = pd.DataFrame()
for index, row in sked_summary.iterrows():
    temp = top_players_by_team.loc[(top_players_by_team['team'] == index) & (top_players_by_team['secondaryPosition'] == 'W')]
    temp = temp.loc[temp['available'] >= 25]
    temp['W_rank'] = row['W_rank'].astype(int)
    w_adds = pd.concat([w_adds, temp])
    if row['W_rank'] == 5:
        break
        
sked_summary.sort_values('G_rank', ascending=True, inplace=True)
g_adds = pd.DataFrame()
for index, row in sked_summary.iterrows():
    temp = summary_stats_last5G.loc[(summary_stats_last5G['team'] == index)]
    temp = temp.loc[temp['available'] >= 25]
    temp['G_rank'] = row['G_rank'].astype(int)
    g_adds = pd.concat([g_adds, temp])
    if row['G_rank'] == 5:
        break
        
adds = pd.concat([d_adds, c_adds, w_adds])
adds['merged_rank'] = adds['D_rank'].combine_first(adds['C_rank']).combine_first(adds['W_rank'])
adds = adds.drop(['D_rank', 'C_rank', 'W_rank'], axis=1)
sked_stats

for index, row in adds.iterrows(): 
#     print(f"<p><b>{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]}, {row['position']}, {team_names_only.loc[team_names_only['abbreviation'] == row['team']]['name'].iloc[0]} </b>({round(row['available'], 2)}% available): {round(row['fantasyPoints'], 2)} fantasy points ({row['FPPG']} FPPG)")
    print(
    f"<p><b>{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]}, "
    f"{row['position']}, {team_names_only.loc[team_names_only['abbreviation'] == row['team']]['name'].iloc[0]} "
    f"</b>({round(row['available'], 2)}% available): {round(row['fantasyPoints'], 2)} fantasy points "
    f"({row['FPPG']} FPPG) {sked_stats.groupby('focusTeam')['opponent'].sum()[row['team']]}"
)
    
for index, row in g_adds.iterrows(): 
#     print(f"<p><b>{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]}, {row['position']}, {team_names_only.loc[team_names_only['abbreviation'] == row['team']]['name'].iloc[0]} </b>({round(row['available'], 2)}% available): {round(row['fantasyPoints'], 2)} fantasy points ({row['FPPG']} FPPG)")
    print(
    f"<p><b>{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]}, "
    f"{row['position']}, {team_names_only.loc[team_names_only['abbreviation'] == row['team']]['name'].iloc[0]} "
    f"</b>({round(row['available'], 2)}% available): {round(row['fantasyPoints'], 2)} fantasy points "
    f"({row['FPP60']} FPP60) {sked_stats.groupby('focusTeam')['opponent'].sum()[row['team']]}"
)

<p><b>Rasmus Sandin, D, Washington Capitals </b>(80.12% available): 10.3 fantasy points (2.06 FPPG) SJSLAK
<p><b>Niko Mikkola, D, Florida Panthers </b>(93.58% available): 10.4 fantasy points (2.08 FPPG) OTTTOR
<p><b>Gustav Forsling, D, Florida Panthers </b>(51.25% available): 6.3 fantasy points (1.26 FPPG) OTTTOR
<p><b>Anton Lundell, C, Florida Panthers </b>(97.64% available): 6.9 fantasy points (1.38 FPPG) OTTTOR
<p><b>Dylan Strome, C, Washington Capitals </b>(51.11% available): 9.3 fantasy points (1.86 FPPG) SJSLAK
<p><b>Beck Malenstyn, L, Washington Capitals </b>(99.81% available): 9.5 fantasy points (1.9 FPPG) SJSLAK
<p><b>Darcy Kuemper, G, Washington Capitals </b>(82.41% available): 2.8 fantasy points (2.6 FPP60) SJSLAK
<p><b>Charlie Lindgren, G, Washington Capitals </b>(85.32% available): 14.6 fantasy points (4.88 FPP60) SJSLAK
<p><b>Hunter Shepard, G, Washington Capitals </b>(99.95% available): 9.2 fantasy points (9.2 FPP60) SJSLAK
<p><b>Pheonix Copley, G, Los Angeles Kings </b>

In [48]:
sked_summary

,gameID,D_FPAPG,C_FPAPG,W_FPAPG,G_FPAPG,D_rank,C_rank,W_rank,G_rank
focusTeam,,,,,,,,,
VGK,4046040660,20.39,17.28,17.29,5.51,1,2,3,1
WSH,4046040665,19.59,14.82,17.39,4.32,2,5,2,2
LAK,2023020339,8.37,8.03,7.10,3.41,25,18,27,3
SJS,2023020326,8.37,8.03,7.10,3.41,26,19,28,4
SEA,2023020333,9.62,9.78,8.19,3.34,14,7,17,5
BOS,2023020321,9.85,8.08,8.43,3.24,12,16,14,6
MTL,2023020337,9.85,8.08,8.43,3.24,13,17,15,7
NJD,2023020327,8.62,8.25,8.72,3.02,23,13,12,8
CBJ,4046040658,16.80,17.35,15.76,2.95,6,1,6,9


In [49]:
lookahead_sked = lookahead_sked.sort_values('gameTime')
lookahead_sked

,gameID,gameDate,gameTime,awayTeam,homeTeam,gameDT,winningGoalie
7,2023020322,2023-11-27,Monday 07:00 PM,BUF,NYR,2023-11-28 00:00:00,0
1118,2023020321,2023-11-27,Monday 07:00 PM,BOS,CBJ,2023-11-28 00:00:00,0
1078,2023020323,2023-11-27,Monday 07:00 PM,FLA,OTT,2023-11-28 00:00:00,0
214,2023020324,2023-11-27,Monday 09:00 PM,TBL,COL,2023-11-28 02:00:00,0
991,2023020325,2023-11-27,Monday 09:30 PM,VGK,CGY,2023-11-28 02:30:00,0
1283,2023020326,2023-11-27,Monday 10:30 PM,WSH,SJS,2023-11-28 03:30:00,0
501,2023020329,2023-11-28,Tuesday 07:00 PM,FLA,TOR,2023-11-29 00:00:00,0
953,2023020327,2023-11-28,Tuesday 07:00 PM,NYI,NJD,2023-11-29 00:00:00,0
584,2023020328,2023-11-28,Tuesday 07:30 PM,CAR,PHI,2023-11-29 00:30:00,0
790,2023020331,2023-11-28,Tuesday 08:00 PM,PIT,NSH,2023-11-29 01:00:00,0


In [50]:
for index, row in lookahead_sked.iterrows():
    
    date_object_utc = datetime.strptime(row['gameDT'], "%Y-%m-%d %H:%M:%S").replace(tzinfo=pytz.UTC)
    date_object_et = date_object_utc.astimezone(pytz.timezone("US/Eastern"))
    formatted_time = date_object_et.strftime("%I:%M p.m.").lstrip('0').replace("PM", "p.m.")
    
    date_string_et = date_object_et.strftime("%Y-%m-%d %H:%M:%S%z")
    date_object = datetime.strptime(date_string_et, "%Y-%m-%d %H:%M:%S%z")    
    day_of_week = date_object.strftime("%A")
    
    home_abbr = row['homeTeam']
    home_top5 = top_players_by_team.loc[top_players_by_team['team'] == home_abbr]
    home_goalies = summary_stats_last5G.loc[summary_stats_last5G['team'] == home_abbr]

    away_abbr = row['awayTeam']
    away_top5 = top_players_by_team.loc[top_players_by_team['team'] == away_abbr]
    away_goalies = summary_stats_last5G.loc[summary_stats_last5G['team'] == away_abbr]
    
    home_team = team_names_only.loc[team_names_only['abbreviation'] == home_abbr]['name'].iloc[0]
    away_team = team_names_only.loc[team_names_only['abbreviation'] == away_abbr]['name'].iloc[0]
        
    print(f"\033[1m{away_team} at {home_team}\033[0m")
    print(f"{formatted_time}")
    print()
    print(f"\033[1mForwards against {home_abbr} score\033[0m: {opp_forward.loc[opp_forward['abbreviation'] == home_abbr, 'FPAPG'].values[0]} FPAPG, Rank: {opp_forward.loc[opp_forward['abbreviation'] == home_abbr, 'FPAPG_Rank'].values[0]}")
    print(f"\033[1mDefence against {home_abbr} score\033[0m: {opp_defense.loc[opp_defense['abbreviation'] == home_abbr, 'FPAPG'].values[0]} FPAPG, Rank: {opp_defense.loc[opp_defense['abbreviation'] == home_abbr, 'FPAPG_Rank'].values[0]}")
    print(f"\033[1mTop {away_abbr} Players Fantasy Points Last 5 \033[0m")
    for index, row in away_top5.iterrows():
        print(f"<p><b>{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]}, {row['position']}, {away_team} </b>({round(row['available'], 2)}% available): {round(row['fantasyPoints'], 2)} fantasy points ({row['FPPG']} FPPG)")
    print()
    print(f"\033[1mGoalies against {home_abbr} score\033[0m: {gopponent_stats.loc[gopponent_stats['name'] == home_team, 'FPAPG'].values[0]} FPAPG, Rank: {gopponent_stats.loc[gopponent_stats['name'] == home_team, 'FPAPG_Rank'].values[0]}")
    for index, row in away_goalies.iterrows():
        print(f"{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]} ({round(row['available'], 2)}% available) last five: {row['start']} starts, FPP60: {row['FPP60']}, FP: {round(row['fantasyPoints'], 2)}")
    print()
    print(f"\033[1mForwards against {away_abbr} score\033[0m: {opp_forward.loc[opp_forward['abbreviation'] == away_abbr, 'FPAPG'].values[0]} FPAPG, Rank: {opp_forward.loc[opp_forward['abbreviation'] == away_abbr, 'FPAPG_Rank'].values[0]}")
    print(f"\033[1mDefence against {away_abbr} score\033[0m: {opp_defense.loc[opp_defense['abbreviation'] == away_abbr, 'FPAPG'].values[0]} FPAPG, Rank: {opp_defense.loc[opp_defense['abbreviation'] == away_abbr, 'FPAPG_Rank'].values[0]}")
    print(f"\033[1mTop {home_abbr} Players Fantasy Points Last 5 \033[0m")
    for index, row in home_top5.iterrows():
        print(f"<p><b>{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]}, {row['position']}, {home_team}</b>({round(row['available'], 2)}% available): {round(row['fantasyPoints'], 2)} fantasy points ({row['FPPG']} FPPG)")

    print()
    print(f"\033[1mGoalies against {away_abbr} score\033[0m: {gopponent_stats.loc[gopponent_stats['name'] == away_team, 'FPAPG'].values[0]} FPAPG, Rank: {gopponent_stats.loc[gopponent_stats['name'] == away_team, 'FPAPG_Rank'].values[0]}")
    for index, row in home_goalies.iterrows():
        print(f"{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]} ({round(row['available'], 2)}% available) last five: {row['start']} starts, FPP60: {row['FPP60']}, FP: {round(row['fantasyPoints'], 2)}")
    print()
    print('-'*25)
    print()

Buffalo Sabres at New York Rangers
7:00 p.m.

Forwards against NYR score: 12.94 FPAPG, Rank: 32
Defence against NYR score: 7.88 FPAPG, Rank: 31
Top BUF Players Fantasy Points Last 5 
<p><b>Rasmus Dahlin, D, Buffalo Sabres </b>(0.12% available): 16.4 fantasy points (3.28 FPPG)
<p><b>Alex Tuch, R, Buffalo Sabres </b>(14.13% available): 11.6 fantasy points (2.32 FPPG)
<p><b>Jeff Skinner, L, Buffalo Sabres </b>(21.62% available): 11.4 fantasy points (2.28 FPPG)
<p><b>JJ Peterka, R, Buffalo Sabres </b>(90.12% available): 10.1 fantasy points (2.02 FPPG)
<p><b>Owen Power, D, Buffalo Sabres </b>(62.25% available): 7.1 fantasy points (1.42 FPPG)

Goalies against NYR score: -0.31 FPAPG, Rank: 30
Eric Comrie (99.72% available) last five: 2 starts, FPP60: -6.2, FP: -8.2
Ukko-Pekka Luukkonen (94.07% available) last five: 2 starts, FPP60: 5.51, FP: 11.0
Devon Levi (79.16% available) last five: 1 starts, FPP60: -2.53, FP: -4.4

Forwards against BUF score: 15.88 FPAPG, Rank: 17
Defence against BUF sco

In [51]:
temp_team_df = pd.DataFrame()

for index, row in team_names_only.iterrows():
    temp_all = all_df.loc[all_df['team'] == row['abbreviation']].sort_values('gameDate', ascending=False)
    five_dates = temp_all['gameDate'].unique()[:6]
    temp_filtered = temp_all[temp_all['gameDate'].isin(five_dates)]
    temp_team_df = pd.concat([temp_team_df, temp_filtered])
    
temp_team_df = temp_team_df.loc[(temp_team_df['team'] == 'STL') & (temp_team_df['position'] == 'D')]
temp_team_df = summary_statistics(temp_team_df)
# temp_team_df['powerPlayToi'] = f"{(temp_team_df['powerPlayToi'] // 60)}:{(temp_team_df['powerPlayToi'] % 60)}"
temp_team_df

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,hits,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG
3,8475753,J. Faulk,STL,D,7707,6,0,3,1,11,8,14,823,554,-6,12.4,D,D,5.79,2.07
4,8476792,T. Krug,STL,D,7386,6,0,3,2,22,4,8,1159,173,-7,10.6,D,D,5.17,1.77
2,8475181,N. Leddy,STL,D,8116,6,1,1,1,8,2,11,21,911,-5,10.0,D,D,4.44,1.67
6,8481006,T. Tucker,STL,D,884,1,0,0,0,0,1,3,0,0,1,1.6,D,D,6.52,1.60
5,8476892,C. Parayko,STL,D,8566,6,1,0,0,14,5,8,53,981,-5,7.9,D,D,3.32,1.32
7,8481059,S. Perunovich,STL,D,4660,5,0,2,1,5,1,4,1037,0,0,5.1,D,D,3.94,1.02
0,8474145,R. Bortuzzo,STL,D,779,1,0,0,0,0,2,1,0,104,1,0.7,D,D,3.23,0.70
1,8474618,M. Scandella,STL,D,3806,5,0,1,0,3,2,4,17,408,4,3.5,D,D,3.31,0.70


In [52]:
temp_player = all_df.loc[all_df['name'] == 'A. Svechnikov']
temp_player = temp_player.sort_values('gameDate', ascending=True).head(10)
summary_statistics(temp_player)

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,hits,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG
0,8480830,A. Svechnikov,CAR,R,9928,10,0,6,1,21,23,3,1085,4,0,12.4,W,F,4.5,1.24


In [53]:
player_bios.sort_values('fantasyPoints', ascending=False, inplace=True)
player_bios['rank'] = player_bios['fantasyPoints'].rank(ascending=False, method='first')
player_bios.loc[player_bios['team'] == 'NYR']

,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,missedGames,...,fantasyPointsRemain,fullName,birthDate,shootsCatches,height,weight,roster_percent,default_pos,espnId,rank
9,8478550,A. Panarin,NYR,L,22080,19,55.2,9.00,2.91,0,...,183.33,Artemi Panarin,1991-10-30,R,72,175,98.097874,LW,3891952,19.0
18,8476885,J. Trouba,NYR,D,25890,19,51.3,7.13,2.70,0,...,170.10,Jacob Trouba,1994-02-26,R,75,209,97.375897,D,2976839,30.0
30,8475184,C. Kreider,NYR,L,21313,19,48.4,8.18,2.55,0,...,160.65,Chris Kreider,1991-04-30,L,75,233,88.478885,LW,5833,37.0
83,8471734,J. Quick,NYR,G,27196,8,41.0,5.43,5.12,0,...,134.84,Jonathan Quick,1986-01-21,L,73,215,30.380397,G,3634,92.0
100,8476389,V. Trocheck,NYR,C,22777,19,38.6,6.10,2.03,0,...,127.89,Vincent Trocheck,1993-07-11,R,71,187,82.158964,C,2563036,110.0
118,8478048,I. Shesterkin,NYR,G,38165,11,36.8,3.47,3.35,0,...,121.03,Igor Shesterkin,1995-12-30,L,73,197,99.615785,G,3151297,134.0
138,8476459,M. Zibanejad,NYR,C,22448,19,35.3,5.66,1.86,0,...,117.18,Mika Zibanejad,1993-04-18,R,74,201,96.698003,C,2562637,153.0
160,8476979,E. Gustafsson,NYR,D,21814,19,33.9,5.59,1.78,0,...,112.14,Erik Gustafsson,1992-03-14,L,73,189,21.142188,D,3891922,167.0
251,8474009,N. Bonino,NYR,C,13758,19,28.6,7.48,1.51,0,...,95.13,Nick Bonino,1988-04-20,L,73,198,0.481755,C,4927,230.0
267,8482073,B. Schneider,NYR,D,18060,19,27.9,5.56,1.47,0,...,92.61,Braden Schneider,2001-09-20,R,75,211,1.033826,D,4697402,251.0


In [54]:
last_five_df_team
hurricanes = last_five_df_team.loc[last_five_df_team['team'] == 'CAR'].sort_values('gameDate')
summary_statistics(hurricanes)

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,hits,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG
18,8482093,S. Jarvis,CAR,C,5930,5,3,4,1,9,7,1,1123,465,4,12.6,C,F,7.65,2.52
16,8480830,A. Svechnikov,CAR,R,5042,5,1,4,1,16,10,2,896,0,2,10.1,W,F,7.21,2.02
0,8470613,B. Burns,CAR,D,6049,5,2,1,1,11,1,6,950,454,4,9.7,D,D,5.77,1.94
5,8476869,B. Skjei,CAR,D,6356,5,1,1,0,9,2,8,119,406,-4,8.1,D,D,4.59,1.62
6,8476882,T. Teravainen,CAR,L,5546,5,1,4,1,8,0,1,1105,286,3,7.8,W,F,5.06,1.56
9,8477488,B. Pesce,CAR,D,6202,5,0,0,0,11,1,13,149,418,-3,7.7,D,D,4.47,1.54
8,8476958,J. Slavin,CAR,D,6324,5,0,3,1,4,2,7,484,472,6,7.6,D,D,4.33,1.52
17,8480835,J. Drury,CAR,C,2639,5,1,2,0,4,0,5,30,113,1,6.9,C,F,9.41,1.38
12,8478427,S. Aho,CAR,C,6140,5,1,3,1,10,1,0,1163,332,4,6.6,C,F,3.87,1.32
13,8478970,J. Chatfield,CAR,D,2879,3,0,2,0,2,2,3,30,45,2,3.9,D,D,4.88,1.30


In [56]:
top_players_by_team.sort_values('fantasyPoints', ascending=False).head(60)

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,...,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG,available
150,8476453,N. Kucherov,TBL,R,6562,5,4,8,8,25,...,1,1280,1,4,23.7,W,F,13.00,4.74,0.12
329,8478402,C. McDavid,EDM,C,6259,5,3,9,4,14,...,6,1125,97,-2,21.7,C,F,12.48,4.34,0.12
299,8478010,B. Point,TBL,C,6265,5,5,6,6,19,...,1,1184,4,5,21.5,C,F,12.35,4.30,0.34
45,8474564,S. Stamkos,TBL,C,5238,5,4,4,6,16,...,3,1232,12,-3,18.4,C,F,12.65,3.68,0.61
254,8477501,V. Nichushkin,COL,R,7129,5,4,4,4,18,...,3,1448,536,4,17.9,W,F,9.04,3.58,27.68
471,8480069,C. Makar,COL,D,7633,5,1,8,3,8,...,10,1399,1008,7,17.6,D,D,8.30,3.52,0.09
515,8480839,R. Dahlin,BUF,D,8202,5,1,5,2,14,...,12,935,294,-4,16.4,D,D,7.20,3.28,0.12
484,8480208,D. Batherson,OTT,R,5339,5,4,3,3,17,...,3,1104,0,-1,16.4,W,F,11.06,3.28,45.68
188,8476887,F. Forsberg,NSH,L,5622,5,4,3,3,21,...,2,1074,2,4,16.3,W,F,10.44,3.26,10.85
195,8476906,S. Gostisbehere,DET,D,6391,5,2,7,4,10,...,4,1429,501,4,16.0,D,D,9.01,3.20,24.76


In [74]:
preNich = all_df.loc[(all_df['name'] == 'M. Hoffman') & (all_df['gameDate'] <= '2023-11-12')]
postNich = all_df.loc[(all_df['name'] == 'M. Hoffman') & (all_df['gameDate'] > '2023-11-12')]

preNichSum = summary_statistics(preNich)
postNichSum = summary_statistics(postNich)

postNichSum

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,hits,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG
0,8474884,M. Hoffman,SJS,C,5534,6,6,1,1,12,3,3,365,0,1,16.5,C,F,10.73,2.75
